# Payoff Cliquet / Ratchet

Payoff cap/planche sur rendement (simplifie).


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import sys
from pathlib import Path
from IPython.display import display, Markdown

_base = Path.cwd().resolve()
for extra in (_base, _base.parent, _base.parent.parent, _base.parent.parent.parent):
    candidate = extra / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
    candidate = extra / "notebooks" / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
plt.style.use('seaborn-v0_8-darkgrid')

from pricing import fetch_spy_history, view_cliquet, payoff_cliquet
close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])
floor_ref, cap_ref = -0.05, 0.1


D:\PythonDProjects\PaperTradingApp\notebooks\scripts\pricing.py:47: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("SPY", period=period, interval=interval, progress=False)
C:\Users\nicol\AppData\Local\Temp\ipykernel_28268\3359970033.py:22: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  spot_ref = float(close_spy.iloc[-1])


## Payoff interactif
S_T, floor/cap, sigma, r, T.


In [ ]:
floor_slider = widgets.FloatSlider(value=0.0, min=-0.5, max=0.5, step=0.01, description='Floor')
cap_slider = widgets.FloatSlider(value=0.1, min=0.0, max=0.5, step=0.01, description='Cap')
output = widgets.Output()

def _update(change=None):
    with output:
        output.clear_output()
        view_dyn = view_cliquet(spot_ref, floor=floor_slider.value, cap=cap_slider.value)
        premium = float(view_dyn.get('premium', 0.0))
        s_grid = view_dyn['s_grid']
        payoff_grid = view_dyn['payoff']
        pnl_grid = view_dyn['pnl']
        payoff_s0 = float(payoff_grid[np.searchsorted(s_grid, spot_ref, side='left')-1])
        pnl_s0 = payoff_s0 - premium
        fig, ax = plt.subplots(figsize=(7,4))
        ax.plot(s_grid, payoff_grid, label='Payoff cliquet')
        ax.plot(s_grid, pnl_grid, label='P&L net', color='darkorange')
        ax.axvline(spot_ref, color='gray', linestyle='--', label=f'S0 = {spot_ref:.2f}')
        ax.axhline(0, color='black', linewidth=0.8)
        ax.legend(loc='best')
        ax.set_xlabel('Spot')
        ax.set_ylabel('Payoff / P&L')
        ax.set_title('Cliquet / Ratchet (approx)')
        plt.show()
        plt.close(fig)
        display(Markdown(f"Prime ~ {premium:.4f}\n\n- Payoff @ S0 = {payoff_s0:.4f}\n- P&L net = {pnl_s0:.4f}"))

for sl in (floor_slider, cap_slider):
    sl.observe(_update, names='value')
_update()
display(widgets.VBox([floor_slider, cap_slider, output]))
